**REAL-TIME POSE DETECTION**

In [1]:
pip install ultralytics opencv-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\jobin\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [ ]:
from ultralytics import YOLO
import yaml
import os
import shutil
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Define Paths
dataset_path = "/kaggle/input/student/data"  # Read-only dataset path
writable_path = "/kaggle/working/student"  # Writable directory

# Step 2: Copy data.yaml to Writable Directory
os.makedirs(writable_path, exist_ok=True)  # Create if it doesn't exist
shutil.copy(f"{dataset_path}/data.yaml", f"{writable_path}/data.yaml")

# Step 3: Modify Paths in the Copied File
yaml_file = f"{writable_path}/data.yaml"

with open(yaml_file, 'r') as f:
    data = yaml.safe_load(f)

data['train'] = os.path.join(dataset_path, 'train/images')
data['val'] = os.path.join(dataset_path, 'valid/images')
data['test'] = os.path.join(dataset_path, 'test/images')

with open(yaml_file, 'w') as f:
    yaml.dump(data, f)

# Step 4: Train Model and Save Training Logs
model = YOLO('yolov8n.pt')  # Load Pretrained YOLOv8 Model
results = model.train(
    data=yaml_file,  # Use modified YAML path
    epochs=100,
    imgsz=640,
    batch=16,
    project="runs/train",  # Custom training directory
    name="yolov8_classroom_pose"  # Name of experiment
)

# Step 5: Validate the Model and Get Accuracy
metrics = model.val()

# Print Accuracy Metrics
print("\n📊 Validation Metrics:")
print(f"mAP@50: {metrics.box.map50:.4f}")
print(f"mAP@50-95: {metrics.box.map:.4f}")
print(f"Precision: {metrics.box.precision:.4f}")
print(f"Recall: {metrics.box.recall:.4f}")

# Step 6: Load Training Logs for Loss Plot
log_file = "runs/train/yolov8_classroom_pose/results.csv"  # Training log path
if os.path.exists(log_file):
    df = pd.read_csv(log_file)
    
    # Plot Training and Validation Loss
    plt.figure(figsize=(10, 5))
    plt.plot(df['epoch'], df['train/box_loss'], label='Train Box Loss', marker='o')
    plt.plot(df['epoch'], df['val/box_loss'], label='Val Box Loss', marker='s')
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss Curve")
    plt.legend()
    plt.grid()
    plt.show()
else:
    print("❌ Training log file not found. Loss curve cannot be plotted.")

# Step 7: Save the Best Model to Kaggle Working Directory
best_model_path = "runs/train/yolov8_classroom_pose/weights/best.pt"
if os.path.exists(best_model_path):
    shutil.copy(best_model_path, "/kaggle/working/best.pt")
    print("\n✅ Best Model Saved at: /kaggle/working/best.pt")
else:
    print("\n❌ Best model file not found. Check training results.")

Executed the below code locally(Jupyter Notebook)

In [1]:
from ultralytics import YOLO
import cv2
import time

class PoseDetector:
    def __init__(self, model_path):
        self.model = YOLO(model_path)
        self.classes = ['Hand Raise', 'Sit', 'Sleep', 'Stand']
        self.prev_time = 0
        
    def detect_webcam(self):
        cap = cv2.VideoCapture(0)
        cv2.namedWindow("Classroom Pose Detection", cv2.WINDOW_NORMAL)  # Make window resizable
        
        # Set initial window size (optional)
        cv2.resizeWindow("Classroom Pose Detection", 1280, 720)  # Width, Height
        
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                print("Camera error!")
                break

            # Calculate FPS
            current_time = time.time()
            fps = 1 / (current_time - self.prev_time)
            self.prev_time = current_time

            # Perform detection
            results = self.model(frame, conf=0.5)
            annotated_frame = results[0].plot()

            # Show FPS
            cv2.putText(annotated_frame, f"FPS: {int(fps)}", (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Maintain aspect ratio while resizing
            h, w = annotated_frame.shape[:2]
            aspect_ratio = w/h
            
            # Update window size (optional)
            cv2.resizeWindow("Classroom Pose Detection", int(720 * aspect_ratio), 720)
            
            cv2.imshow("Classroom Pose Detection", annotated_frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    detector = PoseDetector("best.pt")
    detector.detect_webcam()


0: 480x640 1 Hand Raise, 236.3ms
Speed: 9.0ms preprocess, 236.3ms inference, 15.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Hand Raise, 206.7ms
Speed: 7.2ms preprocess, 206.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Hand Raise, 217.2ms
Speed: 1.8ms preprocess, 217.2ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Hand Raise, 196.8ms
Speed: 0.0ms preprocess, 196.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Hand Raise, 180.1ms
Speed: 0.0ms preprocess, 180.1ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Hand Raise, 228.8ms
Speed: 6.0ms preprocess, 228.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Hand Raise, 242.3ms
Speed: 6.6ms preprocess, 242.3ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 228.9ms
Speed: 8.3ms preprocess, 228.9ms infer